In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import defaultdict
import os

# Set the path to your JSON file here
file_path = "your_data_file.json"  # Change this to your file path

###############################################################################
# 1. LOAD DATA
###############################################################################
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
else:
    # Load the JSON data from file
    try:
        with open(file_path, "r") as f:
            data_log = json.load(f)
        
        if not data_log:
            print("No trials found in data_log.")
        else:
            print(f"Loaded data from: {file_path}")
            print(f"Number of trials found: {len(data_log)}")

            ###############################################################################
            # 2. GATHER UNIQUE GOALS & ASSIGN COLORS
            ###############################################################################
            unique_goals = set()
            for trial in data_log:
                goal = trial.get("goal_reached")
                if goal is not None:
                    unique_goals.add(tuple(goal) if isinstance(goal, list) else goal)  # convert list->tuple for hashing
            unique_goals = sorted(list(unique_goals))  # consistent ordering

            cmap = plt.cm.get_cmap("tab10")
            goal_colors = {}
            for i, g in enumerate(unique_goals):
                color = cmap(i % 10)
                goal_colors[g] = color

            ###############################################################################
            # 3. PLOT RAW TRAJECTORIES
            ###############################################################################
            plt.figure(figsize=(10, 8))
            used_labels = set()

            for trial in data_log:
                trajectory = trial.get("trajectory", [])
                if not trajectory:
                    continue

                # Trajectory is expected as (x, y, gamma)
                xs = [pt[0] for pt in trajectory]
                ys = [pt[1] for pt in trajectory]

                outcome = trial.get("trial_outcome")
                goal = trial.get("goal_reached")

                if outcome == "success" and goal is not None:
                    goal_tuple = tuple(goal) if isinstance(goal, list) else goal
                    line_color = goal_colors.get(goal_tuple, "green")
                    label_text = f"Goal {goal_tuple}"
                else:
                    line_color = "red"
                    label_text = f"Outcome: {outcome}"

                # Add each label only once
                label = label_text if label_text not in used_labels else None
                if label:
                    used_labels.add(label_text)

                plt.plot(xs, ys, color=line_color, linewidth=1.5, alpha=0.8, label=label)

                # Plot start and end points
                plt.scatter(xs[0], ys[0], color='blue', s=100, marker='o', label='Start' if 'Start' not in used_labels else None)
                if 'Start' not in used_labels:
                    used_labels.add('Start')
                
                plt.scatter(xs[-1], ys[-1], color='green', s=100, marker='x', label='End' if 'End' not in used_labels else None)
                if 'End' not in used_labels:
                    used_labels.add('End')

            plt.xlabel("X Coordinate")
            plt.ylabel("Y Coordinate")
            plt.title("Trajectory Path")
            plt.grid(True)
            plt.legend()
            plt.tight_layout()
            plt.show()  # This will display the plot in the notebook

            ###############################################################################
            # 4. COMPUTE QUALITY METRICS
            ###############################################################################
            dot_diameter = 30.0  # Assuming DOT_RADIUS=15

            fitts_metrics = []
            for trial in data_log:
                outcome = trial.get("trial_outcome")
                traj = trial.get("trajectory", [])
                if len(traj) < 2:
                    continue

                # For manual resets, we'll still compute metrics but mark them differently
                T = trial.get("trial_duration", 0)
                if T <= 0:
                    # If duration is 0, estimate it based on average movement speed
                    # Assuming 60 frames per second as a rough estimate
                    T = len(traj) / 60.0  

                traj_np = np.array(traj)
                path_length = 0.0
                for i in range(len(traj_np) - 1):
                    x1, y1 = traj_np[i][0], traj_np[i][1]
                    x2, y2 = traj_np[i+1][0], traj_np[i+1][1]
                    path_length += math.hypot(x2 - x1, y2 - y1)

                start_x, start_y = traj_np[0][0], traj_np[0][1]
                end_x, end_y     = traj_np[-1][0], traj_np[-1][1]
                direct_dist      = math.hypot(end_x - start_x, end_y - start_y)

                if path_length < 1e-9:
                    continue

                # Fitts' ID using path length
                ID = math.log2(path_length / dot_diameter + 1.0)
                throughput = ID / T if T > 0 else float('nan')

                path_efficiency = direct_dist / path_length if path_length > 0 else 0.0
                avg_speed = path_length / T if T > 0 else float('nan')
                overshoot = path_length - direct_dist

                # Simple angle-change measure
                angle_changes = []
                for i in range(1, len(traj_np) - 1):
                    ax = traj_np[i][0] - traj_np[i-1][0]
                    ay = traj_np[i][1] - traj_np[i-1][1]
                    bx = traj_np[i+1][0] - traj_np[i][0]
                    by = traj_np[i+1][1] - traj_np[i][1]
                    mag_a = math.hypot(ax, ay)
                    mag_b = math.hypot(bx, by)
                    if mag_a > 1e-9 and mag_b > 1e-9:
                        dot_ab = ax*bx + ay*by
                        cos_theta = dot_ab / (mag_a*mag_b)
                        cos_theta = max(-1.0, min(1.0, cos_theta))
                        angle = math.acos(cos_theta)
                        angle_changes.append(angle)
                total_angle_change = sum(angle_changes)

                fitts_metrics.append({
                    "mode": trial.get("mode", "Unknown"),  # "Manual" or "AI"
                    "goal_reached": trial.get("goal_reached"),
                    "outcome": outcome,
                    "time": T,
                    "path_length": path_length,
                    "direct_distance": direct_dist,
                    "path_efficiency": path_efficiency,
                    "avg_speed": avg_speed,
                    "overshoot": overshoot,
                    "ID": ID,
                    "throughput": throughput,
                    "angle_change_sum": total_angle_change
                })

            ###############################################################################
            # 5. PRINT METRICS
            ###############################################################################
            if not fitts_metrics:
                print("No trials with valid data to compute metrics.")
            else:
                metrics_by_mode = defaultdict(list)
                for fm in fitts_metrics:
                    metrics_by_mode[fm["mode"]].append(fm)

                print("===== Trajectory Metrics by Mode =====")
                for mode, data_list in metrics_by_mode.items():
                    if not data_list:
                        continue

                    print(f"Mode={mode}")
                    
                    # Group by outcome
                    outcomes = defaultdict(list)
                    for m in data_list:
                        outcomes[m["outcome"]].append(m)
                    
                    for outcome, metrics in outcomes.items():
                        if not metrics:
                            continue
                            
                        time_mean = np.mean([d["time"] for d in metrics])
                        path_len_mean = np.mean([d["path_length"] for d in metrics])
                        eff_mean = np.mean([d["path_efficiency"] for d in metrics])
                        speed_mean = np.mean([d["avg_speed"] for d in metrics if not np.isnan(d["avg_speed"])])
                        throughput_mean = np.mean([d["throughput"] for d in metrics if not np.isnan(d["throughput"])])
                        angle_mean = np.mean([d["angle_change_sum"] for d in metrics])
                        
                        print(f"  Outcome={outcome}:")
                        print(f"    Number of Trials:       {len(metrics)}")
                        print(f"    Average Time:           {time_mean:.2f} s")
                        print(f"    Total Path Length:      {path_len_mean:.2f} px")
                        print(f"    Direct Distance:        {np.mean([d['direct_distance'] for d in metrics]):.2f} px")
                        print(f"    Path Efficiency:        {eff_mean:.2f}")
                        print(f"    Average Speed:          {speed_mean:.2f} px/s")
                        print(f"    Average Throughput:     {throughput_mean:.2f} bits/s")
                        print(f"    Total Angle Change:     {angle_mean:.2f} radians")
                        print("")

            ###############################################################################
            # 6. PLOT GAMMA VALUES ALONG THE TRAJECTORY
            ###############################################################################
            plt.figure(figsize=(10, 8))
            all_x = []
            all_y = []
            all_g = []

            for trial in data_log:
                trajectory = trial.get("trajectory", [])
                for step in trajectory:
                    if len(step) >= 3:
                        x, y, g = step[0], step[1], step[2]
                        all_x.append(x)
                        all_y.append(y)
                        all_g.append(g)

            if all_x and all_y and all_g:
                # Use scatter plot with color representing gamma
                sc = plt.scatter(all_x, all_y, c=all_g, cmap='rainbow', s=30, alpha=0.8,
                                vmin=0.0, vmax=1.0)
                
                # Connect points with a line to show trajectory path
                plt.plot(all_x, all_y, 'k-', alpha=0.3, linewidth=0.5)
                
                plt.colorbar(sc, label='Gamma Value')
                plt.xlabel("X Coordinate")
                plt.ylabel("Y Coordinate")
                plt.title("Gamma Values Along Trajectory")
                plt.grid(True)
                plt.tight_layout()
                plt.show()  # This will display the plot in the notebook
            else:
                print("No gamma data found in trajectories.")

            ###############################################################################
            # 7. GAMMA VALUES OVER TIME
            ###############################################################################
            if all_g:
                plt.figure(figsize=(10, 4))
                time_points = range(len(all_g))
                plt.plot(time_points, all_g, 'b-', linewidth=2)
                plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.7)
                plt.xlabel("Time Step")
                plt.ylabel("Gamma Value")
                plt.title("Gamma Value Over Time")
                plt.grid(True)
                plt.ylim(-0.05, 1.05)  # Add a bit of padding
                plt.tight_layout()
                plt.show()  # This will display the plot in the notebook
                
                # Calculate statistics on gamma
                gamma_np = np.array(all_g)
                print("===== Gamma Statistics =====")
                print(f"Mean Gamma:      {np.mean(gamma_np):.4f}")
                print(f"Max Gamma:       {np.max(gamma_np):.4f}")
                print(f"Min Gamma:       {np.min(gamma_np):.4f}")
                print(f"Std Dev Gamma:   {np.std(gamma_np):.4f}")
                
                # Count transitions between high/low gamma
                threshold = 0.5
                transitions = 0
                for i in range(1, len(gamma_np)):
                    if (gamma_np[i-1] < threshold and gamma_np[i] >= threshold) or \
                       (gamma_np[i-1] >= threshold and gamma_np[i] < threshold):
                        transitions += 1
                
                print(f"Gamma Transitions (threshold={threshold}): {transitions}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file: {file_path}")
    except Exception as e:
        print(f"Error reading file: {e}")

usage: ipykernel_launcher.py [-h] file_path
ipykernel_launcher.py: error: the following arguments are required: file_path


SystemExit: 2